In [ ]:
import qibo
from qibo import gates, Circuit

In [ ]:
qibo.set_backend("numpy")

In [ ]:
circuit = Circuit(4)
circuit.add(gates.X(0))
circuit.add(gates.CNOT(0, 1))

circuit.draw()

In [ ]:
from qibo.hamiltonians import SymbolicHamiltonian
from qibo.symbols import I, X, Y, Z

In [ ]:
some_term = Y(0)*Z(1)*Y(2)*Z(3)

In [ ]:
some_hamiltonian = SymbolicHamiltonian(some_term)

In [ ]:
from qibochem.measurement import expectation, expectation_from_samples

In [ ]:
result = expectation(circuit, some_hamiltonian)
result

In [ ]:
z0_ham = SymbolicHamiltonian(Z(0)*I(3))

z0_exp = expectation(circuit, z0_ham)
z0_exp

In [ ]:
x0_ham = SymbolicHamiltonian(X(0)*I(3))

x0_exp = expectation(circuit, x0_ham)
x0_exp

## State vector of a qubit

The actual state vector of a single qubit (formally, the wave function for the system) can be given by a column vector, e.g. 

$$
\begin{pmatrix}
1 \\ 0
\end{pmatrix}
$$

The state vector for multiple qubits can be found by taking the tensor product ($\otimes$) of the individual vectors:

$$
\begin{pmatrix}
1 \\ 0
\end{pmatrix}
\otimes
\begin{pmatrix}
1 \\ 0
\end{pmatrix}
=
\begin{pmatrix}
1 * \begin{pmatrix}
1 \\ 0
\end{pmatrix} \\
0 * \begin{pmatrix}
1 \\ 0
\end{pmatrix}
\end{pmatrix}
= 
\begin{pmatrix}
1 \\ 0 \\ 0 \\0
\end{pmatrix}
$$

## Bra-ket or Dirac notation

Bra-ket notation is a simpler form to denote the state vector of the system; "bra" refers to a row vector, and "ket" is a column vector.

An example for a 1-qubit system:
$$
\text{bra: }
\langle 0 \rvert =
\begin{pmatrix}
1 & 0
\end{pmatrix}
\text{, and ket: }
\lvert 0 \rangle =
\begin{pmatrix}
1 \\ 0
\end{pmatrix}
$$

and

$$
\text{bra: }
\langle 1 \rvert =
\begin{pmatrix}
0 & 1
\end{pmatrix}
\text{, and ket: }
\lvert 1 \rangle =
\begin{pmatrix}
0 \\ 1
\end{pmatrix}
$$

Same thing for multiple qubits:

$$
\lvert 01 \rangle =
\lvert 0 \rangle \otimes \lvert 1 \rangle =
\begin{pmatrix}
0 \\ 1 \\ 0 \\ 0
\end{pmatrix}
$$

Another way of seeing it is, the binary string ("01") in the ket refers to which row is 1 in the column matrix.

E.g. converting "11" from binary to decimal gives $1*2^{1} + 1*2^{0} = 3$, so the row 3 is 1 in $\lvert 11 \rangle$.
(Python indexing starts from 0, so row 3 is the last row in our 4 by 1 column vector.)

# Some linear algebra

In general, the state vector can be written as a linear combination of $\lvert 0 \rangle$ and $\lvert 1 \rangle$ (basis functions):

$$
\text{General state vector, }
\lvert \Psi \rangle
=
\begin{pmatrix}
a \\ b
\end{pmatrix}
=
a\begin{pmatrix}
1 \\ 0
\end{pmatrix}
+
b\begin{pmatrix}
0 \\ 1
\end{pmatrix}
=
a \lvert 0 \rangle + b\lvert 1 \rangle
$$

where $a$ and $b$ are some real numbers

## Applying it to quantum measurements

A basic [rule](https://en.wikipedia.org/wiki/Born_rule) of quantum mechanics is $a^{2} + b^{2} = 1$.
The squared coefficients of each basis function is the probability of obtaining that particular measurement.

E.g. for the $Z$ term, if our qubit is in the state $\lvert 0 \rangle$, we will get $+1$ when we measure $Z$, and $-1$ if it is in the state $\lvert 1 \rangle$.

For the general case $\Psi = a \lvert 0 \rangle + b\lvert 1 \rangle$, the mean (formally, the expectation) value of $Z$ will be $(1*a^{2} + (-1)*b^{2}) = a^{2} - b^{2}$

If we don't know the exact coefficients $a$ and $b$. we have to take a number of measurements for $Z$ separately, giving us something like: `sample_result = [+1, -1, +1, +1, ..., +1]`.
Ideally, the mean value of `sample_result` will be close to the exact value of $a^{2} - b^{2}$ as we obtain more and more measurements.

In [ ]:
circuit = Circuit(1)
circuit.add(gates.RX(0, 0.3))

circuit.draw()

In [ ]:
z0_ham2 = SymbolicHamiltonian(Z(0))

In [ ]:
expectation_from_samples(circuit, z0_ham2)

In [ ]:
expectation(circuit, z0_ham2)

In [ ]:
expectation_from_samples(circuit, z0_ham2, n_shots=1000000)

In [ ]:
hamiltonian = SymbolicHamiltonian(0.5*X(0)*X(1) + 5*Y(0)*Y(1) + 25*Z(0)*Z(1))

In [ ]:
circuit = Circuit(2)
circuit.add(gates.RX(_i, 0.3) for _i in range(2))

circuit.draw()

In [ ]:
expectation(circuit, hamiltonian)

In [ ]:
expectation_from_samples(circuit, hamiltonian)

In [ ]:
expectation_from_samples(circuit, hamiltonian, n_shots_per_pauli_term=False, shot_allocation=[10, 100, 100000])

In [ ]:
x_ham = SymbolicHamiltonian(0.5*X(0)*X(1))

k = 1000
samples = [expectation_from_samples(circuit, x_ham, n_shots=1) for _ in range(k)]
samples[:5]

In [ ]:
bell_circuit = circuit.copy()
bell_circuit.add(gates.CNOT(1, 0))
bell_circuit.add(gates.H(1))

bell_circuit.draw()

In [ ]:
circuit_result = bell_circuit()
probabilities = circuit_result.probabilities()

In [ ]:
probabilities

In [ ]:
phi_p, phi_m, psi_p, psi_m = probabilities

In [ ]:
sigma_xx = 0.5*(phi_p - phi_m + psi_p - psi_m)
sigma_yy = 5*(-phi_p + phi_m + psi_p - psi_m)
sigma_zz = 25*(phi_p + phi_m - psi_p - psi_m)

print(sigma_xx + sigma_yy + sigma_zz)